In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

import numpy as np
from log import log

In [2]:
from region import get_region_by_two_bases, clean_region

In [3]:
import datetime
from utils import get_path, DATETIME_FORMAT

In [4]:
import seaborn as sns

In [5]:
sql = '''
select a.id appoint_id,
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case when pc.product_id in (
            select id from product
            where full_name like '%ВЭБ%') then 'VEBMED'
        	else 'ПРОЧИЕ'
        end "name",
	   DATE(a.finished + 3 * interval '1 hour')
from appointment a
left join patient p on a.patient_id = p.id 
left join person per on p.person_id = per.id 
left join promotion promo on a.promotion_id = promo.id
----
inner join product_condition pc
   on promo.product_condition_id = pc.id
----
where a.finished at time zone 'UTC' > '2020-03-30'
and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second')
and a.good
	and not (
			a.patient_id in (
							select patient_id from patient_categories pcat
							inner join reference rf 
							on rf.id = pcat.reference_id
							where rf.code = 'TEST'
			)
			or lower(a.report_comment) = 'тест'
	)

'''

In [6]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)

2020-05-28 13:25:02 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-05-28 13:25:02 creating ssh tunnel to 172.16.100.19 as root...
2020-05-28 13:25:02 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 38515
2020-05-28 13:25:02 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [7]:
data = connection.query(sql)

2020-05-28 13:25:02 @telemed query:
                    select a.id appoint_id,
				    	   p.person_id,
				    	   per.phone,
				    	   --per.partner_id,
				    	   --par."name", 
				    	   case when pc.product_id in (
				                select id from product
				                where full_name like '%ВЭБ%') then 'VEBMED'
				            	else 'ПРОЧИЕ'
				            end "name",
				    	   DATE(a.finished + 3 * interval '1 hour')
				    from appointment a
				    left join patient p on a.patient_id = p.id 
				    left join person per on p.person_id = per.id 
				    left join promotion promo on a.promotion_id = promo.id
				    ----
				    inner join product_condition pc
				       on promo.product_condition_id = pc.id
				    ----
				    where a.finished at time zone 'UTC' > '2020-03-30'
				    and a.finished < timezone('UTC', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * int

In [8]:
connection.close()

2020-05-28 13:25:06 @telemed: closing postgres connection...
2020-05-28 13:25:06 closing ssh tunnel to 172.16.100.19...


In [9]:
df = pd.DataFrame(data).copy()

In [10]:
df.head()

,appoint_id,person_id,phone,name,date
0,137211,1585718,79091582287,ПРОЧИЕ,2020-04-08
1,127919,2546091,79518565014,VEBMED,2020-03-31
2,144566,3521114,79687101977,VEBMED,2020-04-15
3,136013,3201622,79089255484,ПРОЧИЕ,2020-04-07
4,128137,1141636,79180989263,ПРОЧИЕ,2020-03-31


In [11]:
# df_test = pd.DataFrame(data).copy()

In [12]:
# df_test['region'] = df_test['phone'].apply(get_region_by_phone_nikita)

In [13]:
# df_test[df_test['region'] != 'Неизвестный регион']

In [14]:
# df_test.region.unique()

In [15]:
#def phone_base_nikita():
#    base = pd.read_csv('phone_city_map.csv')
#    
#    column_base = ['idx', 'phone', 'region']
#    base['phone'] = base['phone'].astype(str)
#    base.columns = column_base
#    base['clean_region'] = ''
#    
#    for i in range(base.shape[0]):
#        temp = base['region'][i].replace('(', '').replace(')', '').replace("'"," ")
#        temp = list(temp.split(','))
#        for j in range(len(temp)):
#            temp[j] = temp[j].strip()
#            
#        temp = [x for x in temp if x]
#        if not temp:
#            temp = ['Заглушка','Неизвестный регион']
#        elif len(temp) == 1:
#            temp.insert(0, 'Заглушка')
#        
#        base['clean_region'][i] = temp
#    
#    return base

----------

In [16]:
#def phone_base():
#    base = pd.read_csv('DEF.csv')
#    
#    column_base = ['code', 'start', 'end', 'capacity', 'operator', 'region']
#    base.columns = column_base
#    
#    return base

In [17]:
#def get_region_by_phone(phone, base=phone_base()):
#    if type(phone) != str:
#        phone = str(phone)
#    phone = ''.join(i for i in phone if i.isdigit())
#    if phone[0] == '8' or phone[0] == '7':
#        phone = phone[1:]
#    if len(phone) != 10:
#        return 'Неизвестный регион'
#    return base.loc[(base['code'] == int(phone[0:3])) &
#                    (base['start'] <= int(phone[3:])) &
#                    (base['end'] >= int(phone[3:]))
#                    ]['region'].to_string(index=False).strip()

In [18]:
#def get_region_by_phone_nikita(phone, base=phone_base_nikita()):
#    if type(phone) != str:
#        phone = str(phone)
#    phone = ''.join(i for i in phone if i.isdigit())    
#    if phone[0] == '8':
#        phone = '7'+ phone[1:]
#    if len(phone) != 11:
#        return 'Неизвестный регион'
#    
#    try:
#        return base.loc[(base['phone'] == phone)]['clean_region'].to_list()[0][1]
#    except:
#        return 'Неизвестный регион'
#

In [19]:
#def get_region_by_two_bases(phone, base=phone_base(), base_n=phone_base_nikita()):
#    if type(phone) != str:
#        phone = str(phone)
#    phone = ''.join(i for i in phone if i.isdigit())    
#    if phone[0] == '8':
#        phone = '7'+ phone[1:]
#    if len(phone) != 11:
#        return 'Неизвестный регион'
#    
#    try:
#        return base_n.loc[(base_n['phone'] == phone)]['clean_region'].to_list()[0][1]
#    except:
#        return base.loc[(base['code'] == int(phone[1:4])) &
#                    (base['start'] <= int(phone[4:])) &
#                    (base['end'] >= int(phone[4:]))
#                    ]['region'].to_string(index=False).strip()

In [20]:
#def clean_region(region, dictionary=regions_dict):
#    if region in dictionary:
#        return dictionary[region]
#    return 'Неизвестный регион'

In [21]:
#base_n = phone_base_nikita()

In [22]:
#base_n.loc[(base_n['phone'] == '79525833000')]['clean_region'].to_list()[0][1]

### DICTIONARY FUNDAMENTALS

In [23]:
# base=phone_base()
# base_n=phone_base_nikita()

# temp = []
# for i in range(base_n.shape[0]):
#     if base_n['clean_region'][i][1] not in temp:
#         temp.append(base_n['clean_region'][i][1])

# file_name = 'phones_dictionary.xls'
# writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
# pd.DataFrame(list(base.region.unique())).to_excel(writer, sheet_name='base1')
# pd.DataFrame(temp).to_excel(writer, sheet_name='base_nikita')

# writer.save()

----------

In [24]:
#pd.DataFrame(list(base.region.unique()))

In [25]:
#########base_n[base_n.applymap(lambda x: x[1] if isinstance(x, list) else x).duplicated('clean_region')]

In [26]:
#pd.DataFrame(temp)

In [27]:
regions_dict = {
                'Алтайский край':'Алтайский край',
                'Амурская обл.':'Амурская область',
                'Архангельская обл.':'Архангельская область',
                'Архангельская область и Ненецкий автономный округ':'Архангельская область',
                'Астраханская обл.':'Астраханская область',
                'Белгородская обл.':'Белгородская область',
                'Брянская обл.':'Брянская область',
                'Владимирская обл.':'Владимирская область',
                'Волгоградская обл.':'Волгоградская область',
                'Волгоградская область':'Волгоградская область',
                'Вологодская обл.':'Вологодская область',
                'Воронежская обл.':'Воронежская область',
                'Воронежская область':'Воронежская область',
                'г. Владимир|Владимирская обл.':'Владимирская область',
                'г. Кострома|р-н Костромской|Костромская обл.':'Костромская область',
                'г. Москва и Московская область':'Москва и Московская область',
                'г. Норильск|Красноярский край':'Красноярский край',
                'г. Псков|Псковская обл.':'Псковская область',
                'г. Санкт - Петербург и Ленинградская область':'Санкт-Петербург',
                'г. Санкт-Петербург и Ленинградская область':'Санкт-Петербург',
                'г. Севастополь':'Севастополь',
                'г. Сочи|Краснодарский край':'Краснодарский край',
                'Еврейская автономная обл.':'Еврейская автономная область',
                'Забайкальский край':'Забайкальский край',
                'Ивановская обл.':'Ивановская область',
                'Иркутская обл.':'Иркутская область',
                'Иркутская область':'Иркутская область',
                'Кабардино-Балкарская Республика':'Кабардино-Балкарская Республика',
                'Калининградская обл.':'Калининградская область',
                'Калужская обл.':'Калужская область',
                'Камчатский край':'Камчатский край',
                'Карачаево-Черкесская Республика':'Карачаево-Черкесская Республика',
                'Кемеровская обл.':'Кемеровская область',
                'Кировская обл.':'Кировская область',
                'Корякский округ|Камчатский край':'Камчатский край',
                'Костромская обл.':'Костромская область',
                'Краснодарский край':'Краснодарский край',
                'Красноярский край':'Красноярский край',
                'Курганская обл.':'Курганская область',
                'Курская обл.':'Курская область',
                'Липецкая обл.':'Липецкая область',
                'Магаданская обл.':'Магаданская область',
                'Мурманская обл.':'Мурманская область',
                'Ненецкий АО':'Ненецкий автономный округ',
                'Нижегородская обл.':'Нижегородская область',
                'Новгородская обл.':'Новгородская область',
                'Новосибирская обл.':'Новосибирская область',
                'Омская обл.':'Омская область',
                'Оренбургская обл.':'Оренбургская область',
                'Орловская обл.':'Орловская область',
                'Пензенская обл.':'Пензенская область',
                'Пермский край':'Пермский край',
                'Приморский край':'Приморский край',
                'Псковская обл.':'Псковская область',
                'р-ны Абзелиловский и Белорецкий|р-ны Абзелиловский и Белорецкий':'Республика Башкортостан',
                'Республика Адыгея':'Республика Адыгея',
                'Республика Алтай':'Республика Алтай',
                'Республика Башкортостан':'Республика Башкортостан',
                'Республика Бурятия':'Республика Бурятия',
                'Республика Дагестан':'Республика Дагестан',
                'Республика Ингушетия':'Республика Ингушетия',
                'Республика Кабардино-Балкарская':'Кабардино-Балкарская Республика',
                'Республика Калмыкия':'Республика Калмыкия',
                'Республика Карачаево-Черкесская':'Карачаево-Черкесская Республика',
                'Республика Карелия':'Республика Карелия',
                'Республика Коми':'Республика Коми',
                'Республика Крым':'Республика Крым',
                'Республика Крым и г. Севастополь':'Севастополь',
                'Республика Марий Эл':'Республика Марий Эл',
                'Республика Мордовия':'Республика Мордовия',
                'Республика Саха /Якутия/':'Республика Саха Якутия',
                'Республика Северная Осетия - Алания':'Республика Северная Осетия — Алания',
                'Республика Татарстан':'Республика Татарстан',
                'Республика Тыва':'Республика Тыва',
                'Республика Удмуртская':'Удмуртская Республика',
                'Республика Хакасия':'Республика Хакасия',
                'Российская Федерация':'Неизвестный регион',
                'Ростовская обл.':'Ростовская область',
                'Ростовская область':'Ростовская область',
                'Рязанская обл.':'Рязанская область',
                'Самарская обл.':'Самарская область',
                'Самарская область':'Самарская область',
                'Саратовская обл.':'Саратовская область',
                'Сахалинская обл.':'Сахалинская область',
                'Свердловская обл.':'Свердловская область',
                'Свердловская область':'Свердловская область',
                'Смоленская обл.':'Смоленская область',
                'Смоленская область':'Смоленская область',
                'Ставропольский край':'Ставропольский край',
                'Сургутский район и г. Сургут':'Ханты-Мансийский автономный округ',
                'Тамбовская обл.':'Тамбовская область',
                'Тверская обл.':'Тверская область',
                'Томская обл.':'Томская область',
                'Томская область':'Томская область',
                'Тульская обл.':'Тульская область',
                'Тюменская обл.':'Тюменская область',
                'Тюменская область':'Тюменская область',
                'Удмуртская Республика':'Удмуртская Республика',
                'Ульяновская обл.':'Ульяновская область',
                'Хабаровский край':'Хабаровский край',
                'Ханты - Мансийский - Югра АО':'Ханты-Мансийский автономный округ',
                'Челябинская обл.':'Челябинская область',
                'Челябинская область':'Челябинская область',
                'Чеченская Республика':'Чеченская Республика',
                'Чувашская Республика':'Чувашская Республика',
                'Чувашская Республика - Чувашия':'Чувашская Республика',
                'Чукотский АО':'Чукотский автономный округ',
                'Ямало-Ненецкий АО':'Ямало-Ненецкий автономный округ',
                'Ярославская обл.':'Ярославская область',
                'Москва':'Москва и Московская область',
                'Московская область':'Москва и Московская область',
                'Алматы':'Алматы',
                'Брестская область':'Брестская область',
                'Витебская область':'Витебская область',
                'Восточно-Казахстанская область':'Восточно-Казахстанская область',
                'город республиканского значения Нур-Султан':'город республиканского значения Нур-Султан',
                'Донецкая область':'Донецкая область',
                'Ереван':'Ереван',
                'Ленинградская область':'Санкт-Петербург',
                'Минск':'Минск',
                'Минская область':'Минская область',
                'муниципалитет Тбилиси':'муниципалитет Тбилиси',
                'Северо-Западный федеральный округ':'Санкт-Петербург',
                'Северо-Казахстанская область':'Северо-Казахстанская область',
                'Центральный федеральный округ':'Москва и Московская область',
                'Киев':'Киев',
                'Magnesia ad Sipylum':'Неизвестный регион',
                'Wattala':'Неизвестный регион',
                'автономное сообщество Андалусия':'Неизвестный регион',
                'автономное сообщество Валенсия':'Неизвестный регион',
                'Англия':'Неизвестный регион',
                'Анкара':'Неизвестный регион',
                'Анталья':'Неизвестный регион',
                'Армавирская область':'Неизвестный регион',
                'Атырауская область':'Неизвестный регион',
                'Берлин':'Неизвестный регион',
                'Будапешт':'Неизвестный регион',
                'Бурса':'Неизвестный регион',
                'Вена':'Неизвестный регион',
                'Вестра-Гёталанд':'Неизвестный регион',
                'Виктория':'Неизвестный регион',
                'Гессен':'Неизвестный регион',
                'Джорджия':'Неизвестный регион',
                'Иль-де-Франс':'Неизвестный регион',
                'Истарская жупания':'Неизвестный регион',
                'Калифорния':'Неизвестный регион',
                'Каталония':'Неизвестный регион',
                'Кыршехир':'Неизвестный регион',
                'Ломбардия':'Неизвестный регион',
                'Мангистауская область':'Неизвестный регион',
                'Марыйский велаят':'Неизвестный регион',
                'Норд-Брабант':'Неизвестный регион',
                'Норд-Холланд':'Неизвестный регион',
                'область Варна':'Неизвестный регион',
                'префектура Токио':'Неизвестный регион',
                'Прованс-Альпы-Лазурный Берег':'Неизвестный регион',
                'провинция Бали':'Неизвестный регион',
                'провинция Кханьхоа':'Неизвестный регион',
                'провинция Онтарио':'Неизвестный регион',
                'Пусан':'Неизвестный регион',
                'Пхукет':'Неизвестный регион',
                'Северный Рейн-Вестфалия':'Неизвестный регион',
                'Стокгольм':'Неизвестный регион',
                'столичный регион Брюссель':'Неизвестный регион',
                'Сулия':'Неизвестный регион',
                'Тоскана':'Неизвестный регион',
                'Утрехт':'Неизвестный регион',
                'Уусимаа':'Неизвестный регион',
                'Цюрих':'Неизвестный регион',
                'штат Мэриленд':'Неизвестный регион',
                'штат Нью-Йорк':'Неизвестный регион',
                'штат Флорида':'Неизвестный регион',
                'эмират Дубай':'Неизвестный регион',
                'эмират Рас-эль-Хайма':'Неизвестный регион'
}

In [28]:
"""
base = pd.read_csv('./data/phone_city_map.csv')

column_base = ['idx', 'phone', 'region']
base['phone'] = base['phone'].astype(str)
base.columns = column_base
base['clean_region'] = ''

"""

"\nbase = pd.read_csv('./data/phone_city_map.csv')\n\ncolumn_base = ['idx', 'phone', 'region']\nbase['phone'] = base['phone'].astype(str)\nbase.columns = column_base\nbase['clean_region'] = ''\n\n"

In [29]:
# base.head()

In [30]:
"""
def split_reg(region):
    temp = region.replace('(', '').replace(')', '').replace("'"," ")
    temp = list(temp.split(','))
    for j in range(len(temp)):
        temp[j] = temp[j].strip()

    temp = [x for x in temp if x]
    if not temp:
        temp = ['Заглушка','Неизвестный регион']
    elif len(temp) == 1:
        temp.insert(0, 'Заглушка')
    return temp
"""

'\ndef split_reg(region):\n    temp = region.replace(\'(\', \'\').replace(\')\', \'\').replace("\'"," ")\n    temp = list(temp.split(\',\'))\n    for j in range(len(temp)):\n        temp[j] = temp[j].strip()\n\n    temp = [x for x in temp if x]\n    if not temp:\n        temp = [\'Заглушка\',\'Неизвестный регион\']\n    elif len(temp) == 1:\n        temp.insert(0, \'Заглушка\')\n    return temp\n'

In [31]:
#base['clean_region'] = base['region'].apply(split_reg)

In [32]:
#base.head()

In [33]:
# base=phone_base()

In [34]:
# base.info()

In [35]:
# base.head()

In [36]:
# base.loc[(base['phone'] == '79262121904')]['clean_region'].to_list()

In [37]:
# base.loc[(base['phone'] == '79670921000')]['clean_region'].to_list()

In [38]:
# base.loc[(base['phone'] == )]['clean_region'].to_list()

In [39]:
# base.loc[(base['phone'] == '79262121904')]['clean_region'].to_list()[0][1]


In [40]:
#df

In [41]:
#df_test[(df_test['date'] == df_test.loc[0]['date']) & (df_test['region'] == 'Алтайский край')]

In [42]:
df['dirty_region'] = df['phone'].apply(get_region_by_two_bases)

In [43]:
#df['region'] = df[df['region'] == 'Неизвестный регион'].apply(get_region_by_phone)

In [44]:
#df = df.replace('Series([], )', 'Неизвестный регион')

In [45]:
df['region'] = df['dirty_region'].apply(clean_region)

In [46]:
df.head(15)

,appoint_id,person_id,phone,name,date,dirty_region,region
0,137211,1585718,79091582287,ПРОЧИЕ,2020-04-08,г. Москва и Московская область,Москва и Московская область
1,127919,2546091,79518565014,VEBMED,2020-03-31,Воронежская обл.,Воронежская область
2,144566,3521114,79687101977,VEBMED,2020-04-15,г. Москва и Московская область,Москва и Московская область
3,136013,3201622,79089255484,ПРОЧИЕ,2020-04-07,Свердловская обл.,Свердловская область
4,128137,1141636,79180989263,ПРОЧИЕ,2020-03-31,Краснодарский край,Краснодарский край
5,135952,3201364,79911116364,VEBMED,2020-04-07,Ставропольский край,Ставропольский край
6,168719,3565800,79520769150,ПРОЧИЕ,2020-04-28,Республика Мордовия,Республика Мордовия
7,173510,3557449,79605280008,VEBMED,2020-04-30,Ярославская обл.,Ярославская область
8,152628,3531222,79536239692,VEBMED,2020-04-21,Орловская обл.,Орловская область
9,208128,3616025,79953064562,VEBMED,2020-05-17,Пензенская обл.,Пензенская область


In [47]:
#df.region.unique()

In [ ]:
# temp = df.region.unique()

# temp = list(temp)
# temp.sort()

# temp

In [ ]:
#df[(df['date'] == '2020-03-30') & (df['region'] == 'Алтайский край')]

In [ ]:
#df.info()

In [ ]:
# reg_names = [name for name in df.region.unique()]

In [ ]:
# reg_names.sort()

In [ ]:
# reg_names.insert(0, 'Всего консультаций')

In [ ]:
#reg_names

In [ ]:
#reg_names.index('Incorrect phone')

In [ ]:
#d = dict()

In [ ]:
#for i in range(df.shape[0]):
#    d[df['date'][i]] = d.get(df['date'][i], [0 for name in reg_names]) 
#    d[df['date'][i]][reg_names.index(df['region'][i])] += 1
#    d[df['date'][i]][0] += 1

In [ ]:
#d

In [ ]:
#report = pd.DataFrame.from_dict(d, orient='index').sort_index()

In [ ]:
#report.columns = reg_names

In [ ]:
#report

In [ ]:
# len(df.phone.unique())

In [48]:
d2 = dict()

In [49]:
dates = [date for date in df.date.unique()]

In [50]:
dates.sort()

In [51]:
dates

[datetime.date(2020, 3, 30),
 datetime.date(2020, 3, 31),
 datetime.date(2020, 4, 1),
 datetime.date(2020, 4, 2),
 datetime.date(2020, 4, 3),
 datetime.date(2020, 4, 4),
 datetime.date(2020, 4, 5),
 datetime.date(2020, 4, 6),
 datetime.date(2020, 4, 7),
 datetime.date(2020, 4, 8),
 datetime.date(2020, 4, 9),
 datetime.date(2020, 4, 10),
 datetime.date(2020, 4, 11),
 datetime.date(2020, 4, 12),
 datetime.date(2020, 4, 13),
 datetime.date(2020, 4, 14),
 datetime.date(2020, 4, 15),
 datetime.date(2020, 4, 16),
 datetime.date(2020, 4, 17),
 datetime.date(2020, 4, 18),
 datetime.date(2020, 4, 19),
 datetime.date(2020, 4, 20),
 datetime.date(2020, 4, 21),
 datetime.date(2020, 4, 22),
 datetime.date(2020, 4, 23),
 datetime.date(2020, 4, 24),
 datetime.date(2020, 4, 25),
 datetime.date(2020, 4, 26),
 datetime.date(2020, 4, 27),
 datetime.date(2020, 4, 28),
 datetime.date(2020, 4, 29),
 datetime.date(2020, 4, 30),
 datetime.date(2020, 5, 1),
 datetime.date(2020, 5, 2),
 datetime.date(2020, 5, 3

In [54]:
for i in range(df.shape[0]):
    d2[df['region'][i]] = d2.get(df['region'][i], [0 for date in dates]) 
    d2[df['region'][i]][dates.index(df['date'][i])] += 1
    #d2[df['region'][i]][0] += 1

In [55]:
#d2

In [56]:
report2 = pd.DataFrame.from_dict(d2, orient='index').sort_index()

In [57]:
report2.columns = dates

In [58]:
report2['mean'] = report2.mean(axis=1).astype(int)

In [59]:
report2['total'] = report2.sum(axis=1)

In [60]:
report2.sort_values(report2.columns[10], ascending=False).head(25)
#report2.sort_values('mean', ascending=False).head(25)

,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,mean,total
Москва и Московская область,150,181,216,219,243,187,197,260,240,277,...,545,568,490,367,313,510,507,419,387,23254
Орловская область,5,3,1,1,5,2,1,3,2,17,...,16,20,19,8,7,14,10,16,15,907
Ивановская область,6,5,7,5,6,5,6,10,42,121,...,33,41,48,23,20,47,37,35,33,1997
Санкт-Петербург,40,53,64,55,55,46,49,57,48,54,...,184,178,121,128,135,153,147,127,86,5202
Неизвестный регион,12,16,41,24,25,37,27,54,51,56,...,25,23,24,23,13,20,25,15,41,2490
Республика Татарстан,10,18,14,12,21,19,53,54,44,46,...,76,93,52,33,56,75,92,83,51,3085
Свердловская область,11,15,20,26,26,24,16,28,24,25,...,79,89,69,21,45,54,48,31,38,2330
Ярославская область,2,3,6,12,7,7,11,16,22,19,...,43,46,22,27,22,23,30,21,21,1283
Краснодарский край,13,27,23,19,28,21,27,36,14,26,...,95,107,72,62,66,76,81,73,47,2869
Республика Башкортостан,15,9,18,10,9,36,33,33,33,27,...,119,138,68,100,82,100,104,69,47,2850


-------

In [ ]:
#report3.loc[['Ивановская обл. VEBMED', 'Ивановская обл. _прочие']]

In [ ]:
#report3.loc[['Орловская обл. VEBMED', 'Орловская обл. _прочие']]

In [ ]:
#report3.loc[['Республика Татарстан VEBMED', 'Республика Татарстан _прочие']]

In [61]:
report2 = report2.sort_values('total', ascending=False)

In [ ]:
#report2.to_excel('app_regions_new.xls')

----

# WEBMED

In [62]:
d3 = dict()

In [63]:
dates = [date for date in df.date.unique()]
dates.sort()

In [64]:
#df['region'][1]

In [65]:
for i in range(df.shape[0]):
    if df['name'][i] == 'VEBMED':
        #region_name = df[]
        d3[df['region'][i] + ' VEBMED'] = d3.get(df['region'][i]  + ' VEBMED', [0 for date in dates]) 
        d3[df['region'][i] + ' VEBMED'][dates.index(df['date'][i])] += 1
    #d2[df['region'][i]][0] += 1
    
    if df['name'][i] != 'VEBMED':
        d3[df['region'][i] + ' _прочие'] = d3.get(df['region'][i] + ' _прочие', [0 for date in dates]) 
        d3[df['region'][i] + ' _прочие'][dates.index(df['date'][i])] += 1

In [66]:
report3 = pd.DataFrame.from_dict(d3, orient='index').sort_index()

In [67]:
report3.columns = dates

In [68]:
report3['total'] = report3.sum(axis=1)

In [69]:
report3

,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,total
Алматы VEBMED,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Алтайский край VEBMED,0,1,2,1,4,1,2,4,3,2,...,69,28,60,35,8,22,27,31,30,1071
Алтайский край _прочие,3,1,1,2,5,3,2,6,3,5,...,1,1,1,3,1,0,0,1,2,90
Амурская область VEBMED,0,0,1,1,2,1,0,0,0,2,...,14,17,14,13,4,7,10,8,2,195
Амурская область _прочие,2,0,0,0,0,0,0,3,2,0,...,1,0,0,0,0,0,1,1,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чукотский автономный округ VEBMED,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
Ямало-Ненецкий автономный округ VEBMED,0,0,1,0,1,1,1,3,1,0,...,4,4,6,2,1,2,3,2,2,106
Ямало-Ненецкий автономный округ _прочие,3,3,2,1,1,1,1,3,2,2,...,0,0,0,0,1,0,0,0,0,59
Ярославская область VEBMED,0,3,4,8,6,5,6,14,21,16,...,33,40,43,21,22,20,22,29,19,1141


In [71]:
report3[report3.index == 'Мурманская область VEBMED']

,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,total
Мурманская область VEBMED,0,0,0,0,3,1,1,2,2,2,...,16,17,17,14,8,9,15,8,7,318


In [72]:
report3[report3.index == 'Красноярский край VEBMED']

,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,total
Красноярский край VEBMED,0,2,4,3,5,5,3,4,9,7,...,52,50,51,30,22,53,55,61,63,1285


In [ ]:
#report3.to_excel('app_regions_new-3.xls')

In [ ]:
#import openpyxl

In [75]:
len(df[(df['region'] == 'Мурманская область') & (df['name'] == 'VEBMED')].person_id.unique())

100

In [77]:
len(df[(df['region'] == 'Красноярский край') & (df['name'] == 'VEBMED')].person_id.unique())

374

In [41]:
report_date: str = datetime.datetime.now().strftime(DATETIME_FORMAT).replace(':', '-')

In [42]:
#
file_name = 'appointments_regions' + report_date + '.xlsx'

In [44]:
def styled_report(data):
    cm = sns.light_palette("green", as_cmap=True)
    return data.style.background_gradient(cmap=cm)

In [105]:
styled_report2 = styled_report(report2)
styled_report3 = styled_report(report3)

In [65]:
# styled_report2

In [73]:
cols = list(styled_report2.columns)

In [79]:
cols[0]

datetime.date(2020, 3, 30)

In [77]:
isinstance(cols[0], datetime.date)

True

In [92]:
# type(styled_report2)

In [103]:
def auto_columns_width(file_name, data, sheet_name, formatting=[]):
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    data.to_excel(writer, sheet_name=sheet_name)
    worksheet = writer.sheets[sheet_name]
    cols = list(data.columns)
    #
    for i in range(len(cols)):
        if isinstance(cols[i], datetime.date):
            cols[i] = cols[i].strftime('%Y-%m-%d')
    # print(cols)
    
    for i, col in enumerate(cols):
        print(i, col)
        worksheet.set_column(first_col=i+1, last_col=i+1, width=(len(col) * 1,5))
        
    writer.save()

In [108]:
# auto_columns_width('1123.xlsx', styled_report2, 'test')

In [97]:
# writer.save()

In [ ]:
file_name = 'appointments_regions' + report_date + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

cm = sns.light_palette("green", as_cmap=True)
styled_report2 = report2.style.background_gradient(cmap=cm)
styled_report2.to_excel(writer, sheet_name='Regions total')
worksheet = writer.sheets['Regions total']
worksheet.set_column('A:A', 50)
worksheet.set_column(1, len(report2.columns), 10)

# styled_report3 = report3.style.apply(lambda x: ['background:lightblue' if 'VEBMED' in x else 'background:lightgray' for x in report3.index], axis=0)
styled_report3 = report3.style.background_gradient(cmap=cm)
styled_report3.to_excel(writer, sheet_name='WEB partition')
worksheet = writer.sheets['WEB partition']
worksheet.set_column('A:A', 50)
worksheet.set_column(1, len(report3.columns), 10)


writer.save()

In [ ]:
#report3.to_excel('test.xlsx', engine='xlsxwriter')

In [ ]:
#writer2 = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')
#report3.style.apply(lambda x: ['background:lightblue' if 'VEBMED' in x else 'background:lightgray' for x in report3.index], axis=0)\
#                    .to_excel(writer2, sheet_name='1')
#writer2.save()

In [ ]:
#report3.style.apply(lambda x: ['background:lightblue' if 'VEBMED' in x else 'background:lightgray' for x in report3.index], axis=0)

In [ ]:
#report3.index

In [ ]:
# report3_temp = pd.DataFrame(report3).copy()

In [ ]:
#cm = sns.light_palette("green", as_cmap=True)

#s = report3_temp.style.background_gradient(cmap=cm)

In [ ]:
# s

In [ ]:
#writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')

In [ ]:
#s.to_excel(writer, sheet_name='Sheet1')

In [ ]:
#worksheet = writer.sheets['Sheet1']

In [ ]:
#workbook  = writer.book

In [ ]:
#format1 = workbook.add_format()
#format1.set_bold()
#format1.set_underline()

In [ ]:
#worksheet.set_column('A:A', 50)
#worksheet.set_column(1, len(report3.columns), 10)

In [ ]:
#writer.save()

In [ ]:
#report3.style.apply(lambda x: ['background:lightblue' if 'VEBMED' in x else 'background:lightgray' for x in report3.index], axis=0)

In [ ]:
#rport3

In [ ]:
#reg_dict = pd.DataFrame.from_dict(regions_dict, orient='index')

In [ ]:
#reg_dict.to_csv('reg_dict.csv')

In [ ]:
#a = pd.read_csv('reg_dict.csv')

In [ ]:
#a.columns = ['key', 'value']

In [ ]:
#a.head()

In [ ]:
#regions_dict = pd.Series(a.value.values,index=a.key).to_dict()

In [ ]:
#regions_dict

In [119]:
global writer
writer = None

In [124]:
def corr_width(x: int):
    if x <= 7:
        return 7
    elif x >=50:
        return 50
    return x

def auto_columns_width(file_name, data, sheet_name, formatting=[]):
    
    # writer
    global writer
    if not writer:
        writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    
    # drop pandas RangeIndex
    if isinstance(data.index, pd.core.indexes.range.RangeIndex):
        data.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        data.to_excel(writer, sheet_name=sheet_name)
    
    # select current worksheet
    worksheet = writer.sheets[sheet_name]
    
    # read columns
    cols = list(data.columns)
    cols_iter = list(data.columns)
    
    # datacolumns to str
    for i in range(len(cols)):
        if isinstance(cols[i], datetime.date):
            cols[i] = cols[i].strftime('%Y-%m-%d')
    
    # if index exists = set k=1
    k = 0
    if data.index.dtype == object:
        k = 1
    
    # setting index length
    worksheet.set_column(0, 0, max(data.index.astype(str).map(len)) * 1.5)
    
    # setting columns length
    for i, col in enumerate(cols, k):
        
        if data[cols_iter[i-k]].dtype == int:
            col_len = corr_width(len(col)) * 1.5
        else:
            col_len = max(corr_width(max(data[cols_iter[i-k]].fillna(0).astype(str).map(len))), len(col))
            col_len *= 1.5
        
        print(i, col, col_len)
        worksheet.set_column(i, i, col_len)
        

In [125]:
auto_columns_width('1234.xlsx', styled_report2, 'rep2')
auto_columns_width('1234.xlsx', styled_report3, 'rep3')

writer.save()

TypeError: 'Styler' object is not subscriptable

In [123]:
styled_report2.columns[0]

datetime.date(2020, 3, 30)

In [114]:
styled_report3.columns

Index([2020-03-30, 2020-03-31, 2020-04-01, 2020-04-02, 2020-04-03, 2020-04-04,
       2020-04-05, 2020-04-06, 2020-04-07, 2020-04-08, 2020-04-09, 2020-04-10,
       2020-04-11, 2020-04-12, 2020-04-13, 2020-04-14, 2020-04-15, 2020-04-16,
       2020-04-17, 2020-04-18, 2020-04-19, 2020-04-20, 2020-04-21, 2020-04-22,
       2020-04-23, 2020-04-24, 2020-04-25, 2020-04-26, 2020-04-27, 2020-04-28,
       2020-04-29, 2020-04-30, 2020-05-01, 2020-05-02, 2020-05-03, 2020-05-04,
       2020-05-05, 2020-05-06, 2020-05-07, 2020-05-08, 2020-05-09, 2020-05-10,
       2020-05-11, 2020-05-12, 2020-05-13, 2020-05-14, 2020-05-15, 2020-05-16,
       2020-05-17,    'total'],
      dtype='object')